# Cover Letter Writer

### Load model

In [49]:
from langchain_openai import ChatOpenAI

BASE_URL = "http://localhost:1234/v1"
MODEL_NAME = "meta-llama-3.1-8b-instruct"


model = ChatOpenAI(base_url=BASE_URL,api_key="xxx",name=MODEL_NAME)

### Research chain

In [50]:
from langchain.prompts import ChatPromptTemplate

messages = [
    (
        "system" , """You're a research agent tasked with gathering information about a given company, 
        exploring its field of activity, its products or services, and its target market. Analyze the company's history, including 
        its creation date, founders and strategic evolution. Identifies the company's major projects, collaborationsand partnerships, 
        as well as any awards it has received. Finally, studies its HR policy, including talent management, corporate culture, recruitment 
        practicesand employee well-being initiatives. You have access to the Tavily search tool to help you in your research."""
     ),
    ("human"," Do this for the company: {company_name}") ,
    ("placeholder", "{agent_scratchpad}")
]
search_prompt_template = ChatPromptTemplate.from_messages(messages)

In [51]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults(max_results=10)
tools = [tavily_tool]

In [52]:
from langchain.agents import AgentExecutor, create_openai_functions_agent

agent = create_openai_functions_agent(model, tools, search_prompt_template)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=False)

In [53]:
from langchain_core.runnables import RunnableLambda

get_output = RunnableLambda(lambda x : x['output'])

agent_chain = agent_executor | get_output

In [54]:
#result = agent_chain.invoke({"company_name": "fieldbox.ai"})
#print(result[:100])

### Write chain

In [55]:
messages = [
    ("system","{prompt}"),
    ("human",""" 
     template : {template}
     profil : {profil} 
     offer_data : {offer} 
     company_data : {company_data}"""),
]
write_prompt_template = ChatPromptTemplate.from_messages(messages)

In [56]:
from utils import load_json,load_txt
import os

DATA_DIR = "../data/"

prompt = load_txt(os.path.join(DATA_DIR,"promptV1.txt"))
offer = ""
profil = load_json(os.path.join(DATA_DIR,"profil.json"))
template = load_txt(os.path.join(DATA_DIR,"templateBrut.txt"))
len(prompt),len(offer),len(profil)

(1151, 0, 3000)

In [57]:
write_prompt_template = write_prompt_template.partial(prompt=prompt,offer=offer,profil=profil,template=template)

In [58]:
from langchain_core.output_parsers import StrOutputParser

write_chain = write_prompt_template | model | StrOutputParser()

In [59]:
#result = write_chain.invoke({"company_data":"no data"})
#print(result)

### Final Chain

In [60]:
final_chain = agent_chain | write_chain

In [ ]:
result = final_chain.invoke({"company_name" : "CapGemini"})

In [ ]:
print(result)

Objet : Candidature pour un poste de Développeur Python à Capgemini

Madame, Monsieur,

Je me permets de vous adresser ma candidature pour le poste de Développeur Python au sein de l'équipe de Capgemini. J'ai été particulièrement séduit par l'opportunité de rejoindre une entreprise qui combine expertise en IT consulting, stratégie et management, avec une approche innovante et audacieuse dans les domaines de la technologie et du numérique.

En tant que diplômé de la deuxième année du Bachelor d'informatique à Gradignan, j'ai développé une solide base en programmation Python ainsi qu'une connaissance approfondie de technologies telles que TensorFlow, PyTorch, GitHub, VScode, Ubuntu et MacOs. Ma passion pour l'apprentissage automatique et le deep learning m'a conduit à développer plusieurs projets intéressants, tels que Large Language Model, Neural Networks Library, Neural Style Transfer, Generative Adversial Network, Cellular Automata 3D Renderer, TradeFormer, Diffusion Model, VAE, Maze 